In [2]:
# aircraft_forecasting_optuna.py

# Importaciones básicas
import sys
import os
from pathlib import Path
import logging
import warnings
warnings.filterwarnings('ignore')

# Añadir el directorio padre al path
sys.path.append(str(Path().cwd().parent))

# Importar bibliotecas de análisis
import numpy as np
import pandas as pd
import optuna
from optuna.samplers import TPESampler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import TimeSeriesSplit
import optuna
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from optuna.visualization import (
    plot_optimization_history,
    plot_param_importances,
    plot_parallel_coordinate,
    plot_slice
)

# Importar módulos personalizados
from models import (
    ModelConfig,
    ATCAircraftDataLoader,
    AircraftDataPreprocessor,
    AircraftFeatureEngineer,
    ARIMAModel,
    ProphetModel,
    RandomForestModel,
    LSTMModel,
    EnsembleModel,
    AircraftForecaster,
    MultiModalDataLoader,
    NewsDataLoader,
    WeatherDataLoader,
    XGBoostModel
)

# Configuración de logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

# Configuración
config = ModelConfig()
RANDOM_STATE = 42

In [30]:
def load_and_prepare_data(config, forecast_horizon=1):
    """Carga y prepara los datos para el entrenamiento."""
    logger.info("Cargando y preparando datos...")
    
    # 1. Cargar datos
    data_loader = ATCAircraftDataLoader(config)
    df = data_loader.load_daily_atc_data()
    
    acids = data_loader.load_daily_acids_data(use_one_hot=True)
    df = pd.merge(df, acids, left_index=True, right_index=True, how='left')

    news_loader = NewsDataLoader(config)
    news = news_loader.load_news_events(feature_type='one_hot')
    df = pd.merge(df, news, left_index=True, right_index=True, how='left')

    weather_loader = WeatherDataLoader(config)
    weather = weather_loader.load_weather_data(
        start_date=df.index.min().strftime('%Y-%m-%d'),
        end_date=df.index.max().strftime('%Y-%m-%d')
    )
    df = pd.merge(df, weather, left_index=True, right_index=True, how='left')
 
    # 2. Preprocesar datos
    preprocessor = AircraftDataPreprocessor(config)
    df_processed = preprocessor.preprocess_daily_data(df)
    
    # 3. Ingeniería de características
    feature_engineer = AircraftFeatureEngineer(config)
    df_featured = feature_engineer.create_features(df_processed)
    df_featured = feature_engineer.create_lagged_target(
        df_featured, 
        forecast_horizon=forecast_horizon
    )
    
    # 4. Preparar datos para modelado
    X, y = feature_engineer.select_features_for_model(df_featured)
    
    logger.info(f"Datos preparados: {len(X)} muestras, {len(X.columns)} características")
    
    return X, y, df_featured


In [31]:
def load_and_prepare_data_multimodal(config, forecast_horizon=1):
    """
    Carga y preprocesa los datos de múltiples fuentes.
    """
    logger.info("Iniciando carga de datos...")
    
    # 1. Cargar datos usando MultiModalDataLoader
    logger.info("Cargando datos A = TC, clima y noticias...")
    multimodal_loader = MultiModalDataLoader(config)
    
    # Cargar datos combinados
    combined_data = multimodal_loader.load_multimodal_data(
        data_type='daily_atc',
        use_one_hot=True,
        include_weather=True,
        include_news=False
    )
    
    # 2. Preprocesar datos
    preprocessor = AircraftDataPreprocessor(config)
    processed_data = preprocessor.preprocess_daily_data(combined_data)
    
    # 3. Crear características adicionales
    feature_engineer = AircraftFeatureEngineer(config)
    featured_data = feature_engineer.create_features(processed_data)
    featured_data = feature_engineer.create_lagged_target(featured_data, forecast_horizon=1)

    # 4. Seleccionar features para modelado
    X, y = feature_engineer.select_features_for_model(featured_data)
    
    logger.info(f"Datos preparados para modelado: {len(X)} registros, {len(X.columns)} características")
    
    return X, y, featured_data

In [32]:
def objective(trial, X, y, algorithm_list=['random_forest', 'prophet', 'lstm', 'arima', 'xgboost']):
    """
    Función objetivo para la optimización con Optuna.
    
    Args:
        trial: Objeto de prueba de Optuna
        X: Características de entrenamiento
        y: Variable objetivo
        
    Returns:
        Error de validación (MAE) a minimizar
    """
    # Selección del modelo
    algorithm = trial.suggest_categorical('algorithm', algorithm_list)

    # Crear una copia de la configuración
    trial_config = ModelConfig()

    if algorithm == 'random_forest':
        # Espacio de búsqueda para Random Forest
        trial_config.models['random_forest'] = {
            'n_estimators': trial.suggest_int('rf_n_estimators', 50, 500, step=50),
            'max_depth': trial.suggest_int('rf_max_depth', 3, 30),
            'min_samples_split': trial.suggest_int('rf_min_samples_split', 2, 20),
            'min_samples_leaf': trial.suggest_int('rf_min_samples_leaf', 1, 10),
            'max_features': trial.suggest_categorical('rf_max_features', ['sqrt', 'log2', None]),
            'bootstrap': trial.suggest_categorical('rf_bootstrap', [True, False]),
            'random_state': RANDOM_STATE
        }
        
        model = RandomForestModel(trial_config)
        
    elif algorithm == 'prophet':
        # Espacio de búsqueda para Prophet
        trial_config.models['prophet'] = {
            'yearly_seasonality': trial.suggest_categorical('prophet_yearly', [True, False]),
            'weekly_seasonality': trial.suggest_categorical('prophet_weekly', [True, False]),
            'daily_seasonality': trial.suggest_categorical('prophet_daily', [True, False]),
            'changepoint_prior_scale': trial.suggest_float('prophet_changepoint_prior_scale', 0.001, 0.5, log=True),
            'seasonality_prior_scale': trial.suggest_float('prophet_seasonality_prior_scale', 0.1, 10, log=True),
            'seasonality_mode': trial.suggest_categorical('prophet_seasonality_mode', ['additive', 'multiplicative']),
            'changepoint_range': trial.suggest_float('prophet_changepoint_range', 0.8, 0.95),
            'n_changepoints': trial.suggest_int('prophet_n_changepoints', 10, 50, step=5)
        }
        
        model = ProphetModel(trial_config)
        
    elif algorithm == 'lstm':
        # Espacio de búsqueda para LSTM
        trial_config.models['lstm'] = {
            'sequence_length': trial.suggest_int('lstm_sequence_length', 7, 30, step=7),
            'hidden_units': trial.suggest_int('lstm_hidden_units', 32, 256, step=32),
            'dropout_rate': trial.suggest_float('lstm_dropout', 0.1, 0.5, step=0.1),
            'epochs': trial.suggest_int('lstm_epochs', 50, 200, step=50),
            'batch_size': trial.suggest_categorical('lstm_batch_size', [16, 32, 64]),
            'learning_rate': trial.suggest_float('lstm_learning_rate', 1e-4, 1e-2, log=True),
            'optimizer': trial.suggest_categorical('lstm_optimizer', ['adam', 'rmsprop']),
            'num_layers': trial.suggest_int('lstm_num_layers', 1, 3)
        }
        
        model = LSTMModel(trial_config)
        
    elif algorithm == 'arima':
        # Espacio de búsqueda para ARIMA
        p = trial.suggest_int('arima_p', 0, 5)
        d = trial.suggest_int('arima_d', 0, 2)
        q = trial.suggest_int('arima_q', 0, 5)
        P = trial.suggest_int('arima_P', 0, 3)
        D = trial.suggest_int('arima_D', 0, 2)
        Q = trial.suggest_int('arima_Q', 0, 3)
        s = 7  # Estacionalidad semanal
        
        trial_config.models['arima'] = {
            'order': (p, d, q),
            'seasonal_order': (P, D, Q, s)
        }
        
        model = ARIMAModel(trial_config)

    elif algorithm == 'xgboost':
        # Espacio de búsqueda para XGBoost
        trial_config.models['xgboost'] = {
            'n_estimators': trial.suggest_int('xgb_n_estimators', 50, 500, step=50),
            'max_depth': trial.suggest_int('xgb_max_depth', 3, 15),
            'learning_rate': trial.suggest_float('xgb_learning_rate', 0.01, 0.3, log=True),
            'subsample': trial.suggest_float('xgb_subsample', 0.6, 1.0),
            'colsample_bytree': trial.suggest_float('xgb_colsample_bytree', 0.6, 1.0),
            'min_child_weight': trial.suggest_int('xgb_min_child_weight', 1, 10),
            'gamma': trial.suggest_float('xgb_gamma', 0, 5),
            'reg_alpha': trial.suggest_float('xgb_reg_alpha', 0, 1),
            'reg_lambda': trial.suggest_float('xgb_reg_lambda', 0, 1),
            'random_state': RANDOM_STATE,
            'n_jobs': -1
        }
        
        model = XGBoostModel(trial_config)

    
    else:
        raise ValueError(f"Tipo de modelo no soportado: {algorithm}")
    
    # Validación cruzada temporal
    tscv = TimeSeriesSplit(n_splits=5)
    scores = []
    
    for train_index, val_index in tscv.split(X):
        X_train, X_val = X.iloc[train_index], X.iloc[val_index]
        y_train, y_val = y.iloc[train_index], y.iloc[val_index]
        
        # Ajustar el modelo
        model.fit(X_train, y_train)
        
        # Predecir
        y_pred = model.predict(X_val)

        # Alinear predicciones con el target y filtrar valores no finitos
        y_pred = np.asarray(y_pred).ravel()

        if len(y_pred) != len(y_val):
            min_len = min(len(y_pred), len(y_val))
            y_pred = y_pred[-min_len:]
            y_val_aligned = y_val.iloc[-min_len:]
        else:
            y_val_aligned = y_val

        valid_mask = np.isfinite(y_pred)
        if not valid_mask.any():
            raise ValueError("Predicciones no válidas: todas son NaN o infinitas")

        y_pred = y_pred[valid_mask]
        y_val_aligned = y_val_aligned.iloc[np.where(valid_mask)[0]]

        # Calcular métricas
        mae = mean_absolute_error(y_val_aligned, y_pred)
        scores.append(mae)
    
    # Devolver el MAE promedio
    return np.mean(scores)


In [33]:
def objective_emsemble(trial, X, y, algorithm_list=['ensemble', 'random_forest', 'prophet', 'lstm', 'arima']):
    """
    Función objetivo para la optimización con Optuna.
    
    Args:
        trial: Objeto de prueba de Optuna
        X: Características de entrenamiento
        y: Variable objetivo
        
    Returns:
        Error de validación (MAE) a minimizar
    """
    # Selección del modelo
    algorithm = trial.suggest_categorical('algorithm', algorithm_list)

    # Crear una copia de la configuración
    trial_config = ModelConfig()
    
    # Configuración común para los modelos base
    rf_params = {
        'n_estimators': trial.suggest_int('rf_n_estimators', 50, 500, step=25),
        'max_depth': trial.suggest_int('rf_max_depth', 3, 30),
        'min_samples_split': trial.suggest_int('rf_min_samples_split', 2, 20),
        'min_samples_leaf': trial.suggest_int('rf_min_samples_leaf', 1, 10),
        'max_features': trial.suggest_categorical('rf_max_features', ['sqrt', 'log2', None]),
        'bootstrap': trial.suggest_categorical('rf_bootstrap', [True, False]),
        'random_state': RANDOM_STATE
    }
    
    prophet_params = {
        'yearly_seasonality': trial.suggest_categorical('prophet_yearly', [True, False]),
        'weekly_seasonality': trial.suggest_categorical('prophet_weekly', [True, False]),
        'daily_seasonality': trial.suggest_categorical('prophet_daily', [True, False]),
        'changepoint_prior_scale': trial.suggest_float('prophet_changepoint_prior_scale', 0.001, 0.5, log=True),
        'seasonality_prior_scale': trial.suggest_float('prophet_seasonality_prior_scale', 0.1, 10, log=True),
        'seasonality_mode': trial.suggest_categorical('prophet_seasonality_mode', ['additive', 'multiplicative']),
        'changepoint_range': trial.suggest_float('prophet_changepoint_range', 0.8, 0.95),
        'n_changepoints': trial.suggest_int('prophet_n_changepoints', 10, 50, step=5)
    }
    
    lstm_params = {
        'sequence_length': trial.suggest_int('lstm_sequence_length', 7, 30, step=7),
        'hidden_units': trial.suggest_int('lstm_hidden_units', 32, 256, step=32),
        'dropout_rate': trial.suggest_float('lstm_dropout', 0.1, 0.5, step=0.1),
        'epochs': trial.suggest_int('lstm_epochs', 50, 200, step=50),
        'batch_size': trial.suggest_categorical('lstm_batch_size', [16, 32, 64]),
        'learning_rate': trial.suggest_float('lstm_learning_rate', 1e-4, 1e-2, log=True),
        'optimizer': trial.suggest_categorical('lstm_optimizer', ['adam', 'rmsprop']),
        'num_layers': trial.suggest_int('lstm_num_layers', 1, 3)
    }
    
    # Parámetros ARIMA
    p = trial.suggest_int('arima_p', 0, 5)
    d = trial.suggest_int('arima_d', 0, 2)
    q = trial.suggest_int('arima_q', 0, 5)
    P = trial.suggest_int('arima_P', 0, 3)
    D = trial.suggest_int('arima_D', 0, 2)
    Q = trial.suggest_int('arima_Q', 0, 3)
    s = 7  # Estacionalidad semanal

    if algorithm == 'random_forest':
        trial_config.models['random_forest'] = rf_params
        model = RandomForestModel(trial_config)
        
    elif algorithm == 'prophet':
        trial_config.models['prophet'] = prophet_params
        model = ProphetModel(trial_config)
        
    elif algorithm == 'lstm':
        trial_config.models['lstm'] = lstm_params
        model = LSTMModel(trial_config)
        
    elif algorithm == 'arima':
        trial_config.models['arima'] = {
            'order': (p, d, q),
            'seasonal_order': (P, D, Q, s)
        }
        model = ARIMAModel(trial_config)
        
    elif algorithm == 'ensemble':
        # Crear configuración para el ensemble
        trial_config = ModelConfig()
        
        # Definir espacio de búsqueda para los pesos
        weights = {
            'arima': trial.suggest_float('ensemble_weight_arima', 0, 1),
            'prophet': trial.suggest_float('ensemble_weight_prophet', 0, 1),
            'random_forest': trial.suggest_float('ensemble_weight_rf', 0, 1),
            'lstm': trial.suggest_float('ensemble_weight_lstm', 0, 1)
        }
        
        # Normalizar pesos para que sumen 1
        total = sum(weights.values())
        weights = {k: v/total for k, v in weights.items()}
        
        # Configurar pesos en la configuración
        trial_config.models['ensemble'] = {'weights': weights}
        
        # Configurar parámetros de los modelos base
        trial_config.models['random_forest'] = rf_params
        trial_config.models['prophet'] = prophet_params
        trial_config.models['lstm'] = lstm_params
        trial_config.models['arima'] = {
            'order': (p, d, q),
            'seasonal_order': (P, D, Q, s)
        }
        
        # Crear el ensemble y añadir modelos base
        ensemble = EnsembleModel(trial_config)
        ensemble.add_model(ARIMAModel(trial_config), weight=weights['arima'])
        ensemble.add_model(ProphetModel(trial_config), weight=weights['prophet'])
        ensemble.add_model(RandomForestModel(trial_config), weight=weights['random_forest'])
        ensemble.add_model(LSTMModel(trial_config), weight=weights['lstm'])
        
        model = ensemble
    
    else:
        raise ValueError(f"Tipo de modelo no soportado: {algorithm}")
    
    # Validación cruzada temporal
    tscv = TimeSeriesSplit(n_splits=5)
    scores = []
    
    for train_index, val_index in tscv.split(X):
        X_train, X_val = X.iloc[train_index], X.iloc[val_index]
        y_train, y_val = y.iloc[train_index], y.iloc[val_index]
        
        # Ajustar el modelo
        model.fit(X_train, y_train)
        
        # Predecir
        y_pred = model.predict(X_val)

        # Alinear predicciones con el target y filtrar valores no finitos
        y_pred = np.asarray(y_pred).ravel()

        if len(y_pred) != len(y_val):
            min_len = min(len(y_pred), len(y_val))
            y_pred = y_pred[-min_len:]
            y_val_aligned = y_val.iloc[-min_len:]
        else:
            y_val_aligned = y_val

        valid_mask = np.isfinite(y_pred)
        if not valid_mask.any():
            raise ValueError("Predicciones no válidas: todas son NaN o infinitas")

        y_pred = y_pred[valid_mask]
        y_val_aligned = y_val_aligned.iloc[np.where(valid_mask)[0]]

        # Calcular métricas
        mae = mean_absolute_error(y_val_aligned, y_pred)
        scores.append(mae)
    
    # Devolver el MAE promedio
    return np.mean(scores)


In [34]:
def optimize_hyperparameters(X, y, objective_fuc, algorithm_list=None, n_trials=50):
    """
    Optimiza los hiperparámetros usando Optuna.
    
    Args:
        X: Características
        y: Variable objetivo
        n_trials: Número de pruebas a realizar
        
    Returns:
        study: Objeto de estudio de Optuna
    """
    # Crear o cargar estudio con almacenamiento persistente
    try:
        # Intentar cargar un estudio existente
        study = optuna.create_study(
            study_name=study_name,
            storage=storage_name,
            load_if_exists=True,
            direction='minimize',
            sampler=TPESampler(seed=RANDOM_STATE)
        )
        logger.info(f"Estudio cargado. Número de trials existentes: {len(study.trials)}")
    except Exception as e:
        # Si no existe, crear uno nuevo
        study = optuna.create_study(
            study_name=study_name,
            storage=storage_name,
            direction='minimize',
            sampler=TPESampler(seed=RANDOM_STATE)
        )
        logger.info("Nuevo estudio creado")
    
    # Función objetivo parcial
    def objective_wrapper(trial):
        if algorithm_list:
            return objective_fuc(trial, X, y, algorithm_list)
        else:
            return objective_fuc(trial, X, y)
        
    
    # Calcular cuántos trials nuevos necesitamos
    remaining_trials = max(0, n_trials - len(study.trials))
    
    if remaining_trials > 0:
        logger.info(f"Iniciando optimización con {remaining_trials} pruebas nuevas...")
        study.optimize(objective_wrapper, n_trials=remaining_trials, show_progress_bar=True)
    else:
        logger.info(f"Ya se han completado {len(study.trials)} trials. No se necesitan más pruebas.")
    
    # Mostrar resultados
    logger.info("\nResumen de la optimización:")
    logger.info(f"Número total de trials: {len(study.trials)}")
    logger.info(f"Mejor valor (MAE): {study.best_value:.4f}")
    logger.info("Mejores parámetros encontrados:")
    for key, value in study.best_params.items():
        logger.info(f"  {key}: {value}")
    
    return study


In [35]:
# Configuración de almacenamiento para Optuna
import sqlite3
from pathlib import Path

# Crear directorio para almacenamiento si no existe
storage_dir = Path("optuna_storage")
storage_dir.mkdir(exist_ok=True)

In [36]:
# Configurar el almacenamiento
storage_name = f"sqlite:///{storage_dir}/aircraft_forecasting_airlinesOneHot_NewOneHot_Weather_h1.db"
study_name = "aircraft_forecasting_study"

In [37]:
"""Función principal para ejecutar la optimización."""
# Cargar y preparar datos
X, y, _ = load_and_prepare_data(config,forecast_horizon=1)

study = optimize_hyperparameters(X, y, objective, n_trials=100)

2026-01-09 14:43:33,038 - __main__ - INFO - Cargando y preparando datos...
2026-01-09 14:43:33,039 - models.data_loader - INFO - Cargando datos diarios ATC desde: data/ATC csvs/atc_dayatcopsummary_202512301506.csv
2026-01-09 14:43:33,049 - models.data_loader - INFO - Datos diarios cargados: 764 registros, columnas: ['arrivals', 'departures', 'overflights', 'nationals', 'unknown', 'total', 'fpp']
2026-01-09 14:43:33,050 - models.data_loader - INFO - Cargando datos diarios ACIDs desde: data/ATC csvs/atc_daylyacids_202512301506.csv
2026-01-09 14:43:34,065 - models.data_loader - INFO - Creando conteo de vuelos para 850 aerolíneas únicas
2026-01-09 14:43:36,990 - models.data_loader - INFO - Features de aerolíneas creadas: 764 registros, 850 columnas.
2026-01-09 14:43:36,992 - models.external_data_loaders - INFO - Cargando datos de eventos desde: data/ATC csvs/eventos.json
2026-01-09 14:43:45,763 - models.external_data_loaders - INFO - Conteo de noticias por tipo creado: 78114 registros, 4 c

  0%|          | 0/100 [00:00<?, ?it/s]

2026-01-09 14:43:46,359 - models.model - INFO - Entrenando Prophet
2026-01-09 14:43:46,373 - cmdstanpy - INFO - Chain [1] start processing
2026-01-09 14:43:46,393 - cmdstanpy - INFO - Chain [1] done processing
2026-01-09 14:43:46,395 - models.model - INFO - Prophet entrenado exitosamente
2026-01-09 14:43:46,436 - models.model - INFO - Entrenando Prophet
2026-01-09 14:43:46,453 - cmdstanpy - INFO - Chain [1] start processing
2026-01-09 14:43:46,469 - cmdstanpy - INFO - Chain [1] done processing
2026-01-09 14:43:46,471 - models.model - INFO - Prophet entrenado exitosamente
2026-01-09 14:43:46,513 - models.model - INFO - Entrenando Prophet
2026-01-09 14:43:46,534 - cmdstanpy - INFO - Chain [1] start processing
2026-01-09 14:43:46,592 - cmdstanpy - INFO - Chain [1] done processing
2026-01-09 14:43:46,594 - models.model - INFO - Prophet entrenado exitosamente
2026-01-09 14:43:48,665 - models.model - INFO - Entrenando Prophet
2026-01-09 14:43:48,691 - cmdstanpy - INFO - Chain [1] start proce

[I 2026-01-09 14:43:49,094] Trial 0 finished with value: 593.6484347877597 and parameters: {'algorithm': 'prophet', 'prophet_yearly': True, 'prophet_weekly': True, 'prophet_daily': True, 'prophet_changepoint_prior_scale': 0.41472250004816347, 'prophet_seasonality_prior_scale': 4.622589001020832, 'prophet_seasonality_mode': 'additive', 'prophet_changepoint_range': 0.8275106764780151, 'prophet_n_changepoints': 20}. Best is trial 0 with value: 593.6484347877597.


2026-01-09 14:43:50,112 - models.model - INFO - ARIMA entrenado exitosamente
2026-01-09 14:43:50,123 - models.model - INFO - Entrenando ARIMA con order=(1, 1, 2), seasonal_order=(3, 0, 2, 7)
2026-01-09 14:43:51,975 - models.model - INFO - ARIMA entrenado exitosamente
2026-01-09 14:43:51,987 - models.model - INFO - Entrenando ARIMA con order=(1, 1, 2), seasonal_order=(3, 0, 2, 7)
2026-01-09 14:43:54,900 - models.model - INFO - ARIMA entrenado exitosamente
2026-01-09 14:43:54,908 - models.model - INFO - Entrenando ARIMA con order=(1, 1, 2), seasonal_order=(3, 0, 2, 7)
2026-01-09 14:43:57,843 - models.model - INFO - ARIMA entrenado exitosamente
2026-01-09 14:43:57,853 - models.model - INFO - Entrenando ARIMA con order=(1, 1, 2), seasonal_order=(3, 0, 2, 7)
2026-01-09 14:44:01,886 - models.model - INFO - ARIMA entrenado exitosamente
2026-01-09 14:44:01,965 - models.model - INFO - Entrenando LSTM con sequence_length=28


...

2026-01-09 14:55:54,745 - models.model - INFO - Random Forest entrenado exitosamente
2026-01-09 14:55:54,787 - models.model - INFO - Entrenando Random Forest con 250 árboles
2026-01-09 14:55:56,122 - models.model - INFO - Random Forest entrenado exitosamente
2026-01-09 14:55:56,161 - models.model - INFO - Entrenando Random Forest con 250 árboles
2026-01-09 14:55:58,804 - models.model - INFO - Random Forest entrenado exitosamente
2026-01-09 14:55:58,843 - models.model - INFO - Entrenando Random Forest con 250 árboles
2026-01-09 14:56:03,114 - models.model - INFO - Random Forest entrenado exitosamente
2026-01-09 14:56:03,157 - models.model - INFO - Entrenando Random Forest con 250 árboles
2026-01-09 14:56:08,820 - models.model - INFO - Random Forest entrenado exitosamente
2026-01-09 14:56:08,920 - models.model - INFO - Entrenando Random Forest con 250 árboles


[I 2026-01-09 14:56:08,869] Trial 52 finished with value: 105.64071455749563 and parameters: {'algorithm': 'random_forest', 'rf_n_estimators': 250, 'rf_max_depth': 27, 'rf_min_samples_split': 17, 'rf_min_samples_leaf': 9, 'rf_max_features': None, 'rf_bootstrap': True}. Best is trial 43 with value: 105.64071455749563.


2026-01-09 14:56:09,479 - models.model - INFO - Random Forest entrenado exitosamente
2026-01-09 14:56:09,520 - models.model - INFO - Entrenando Random Forest con 250 árboles


[W 2026-01-09 14:56:10,443] Trial 53 failed with parameters: {'algorithm': 'random_forest', 'rf_n_estimators': 250, 'rf_max_depth': 28, 'rf_min_samples_split': 19, 'rf_min_samples_leaf': 9, 'rf_max_features': None, 'rf_bootstrap': True} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/home/gabo/Personal/Universidad/04 - Cuarto Año/1er Semestre/Machine Learning/Proyectos/ml-project/.venv/lib/python3.11/site-packages/optuna/study/_optimize.py", line 205, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "/tmp/ipykernel_475473/746797510.py", line 39, in objective_wrapper
    return objective_fuc(trial, X, y)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/tmp/ipykernel_475473/2498613537.py", line 111, in objective
    model.fit(X_train, y_train)
  File "/home/gabo/Personal/Universidad/04 - Cuarto Año/1er Semestre/Machine Learning/Proyectos/ml-project/models/model.py", line 257, in fit
    self.model.fit(

KeyboardInterrupt: 